In [1]:
%pip install langchain langchain_community langchain_chroma

In [3]:
%pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


## Methods to Setup Environmental Variables for API Access

### Method 1: Enter Manually on Run (More Secure)

In [4]:
import getpass
import os

In [5]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = getpass.getpass('LANGCHAIN_API_KEY: ')
os.environ['GROQ_API_KEY'] = getpass.getpass('GROQ_API_KEY: ')

### Method 2: Setup .env file with API keys and Load

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'langchain'

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://llamaimodel.com/requirements/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=('stats-table')
        )
    ),
)
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings

In [ ]:
embedding = OllamaEmbeddings(
    model='llama3.1'
)

In [ ]:
# vectorstore.delete_collection()
# vectorstore.reset_collection()

In [ ]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(model="llama3-8b-8192")

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What's the article about?")

In [ ]:
rag_chain.invoke("what are the requirements for the 8b model?")